# Similarity

In [37]:
###########
# Imports #
##############################################################################

import io
import base64

import numpy as np
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import sliding_window_view

##############################################################################


In [38]:
import os
import json

from rag.db.postgresql_db import PostgreSQLDB
from rag.ai.pattern_ai import PatternAI

from IPython.display import Image, display

import base64

# check_plot

In [49]:
symbol = "ADAUSDT"
time_column="time"
vector_column="embedding"
table_name="market_pattern"

top_k=10

In [50]:
db = PostgreSQLDB()
db.set_connection()

ai = PatternAI(
    symbol=symbol,
    timeframe="15m",
    model="MOCKUP",
    vector_window=60,
)

2026-01-02 16:50:08,372 - PostgreSQLDB - INFO - Initialize database with db_name: prod || user: postgres


Creating UTC logger with level: 20
Creating UTC logger with level: 20
Creating UTC logger with level: INFO
Creating UTC logger with level: INFO


In [ ]:
raw_sql = f"""
    SELECT {vector_column} 
    FROM {table_name} 
    order by random()
    limit 1
"""
db.cursor.execute(raw_sql)
row = db.cursor.fetchone()
target_vec = row[0]

if hasattr(target_vec, 'tolist'):
    target_vec = target_vec.tolist()

In [52]:
if isinstance(target_vec, str):
    actual_vector = json.loads(target_vec)
else:
    actual_vector = target_vec
actual_vector

[0.6535628,
 -0.9280643,
 1.3115144,
 0.5197404,
 0.3876526,
 -0.39962593,
 -0.13725077,
 1.5654603,
 -0.66047615,
 -1.1855297,
 -0.13725077,
 -0.53098565,
 -1.3205262,
 -0.13725077,
 2.3571372,
 -1.0546283,
 0.38719392,
 0.7790596,
 -1.5780058,
 -0.26845747,
 0.25625464,
 -0.39954948,
 -0.13725077,
 0.51820964,
 -0.7927112,
 -0.26845747,
 3.2617545,
 2.2009602,
 0.7687832,
 1.0249766,
 -0.7825621,
 -0.13725077,
 0.3790723,
 0.3784807,
 -1.9449022,
 0.12143004,
 -1.1728672,
 -1.8252261,
 0.38280082,
 0.12254987,
 -0.3970514,
 -1.438508,
 -0.006955614,
 0.123226434,
 -0.39772797,
 1.5531628,
 0.64080405,
 1.0273155,
 -1.1722718,
 -0.2667959,
 -0.78553635,
 0.8995584,
 -0.78497654,
 0.25149655,
 -1.1746573,
 0.12232513,
 0.38145235,
 -0.2668705,
 0.63990843,
 -0.5256626]

In [53]:
query_neighbors = f"""
    SELECT 
        {time_column}, 
        next_return, 
        next_slope_3, 
        next_slope_5,
        {vector_column} <=> '{actual_vector}' as distance,
        {vector_column}
    FROM {table_name}
    WHERE symbol = '{symbol}'
    AND {vector_column} IS NOT NULL
    AND {time_column} < '2026-01-01 00:00:00'::timestamptz
    ORDER BY distance ASC
    LIMIT {top_k}
"""

db.cursor.execute(query_neighbors)

neighbors = db.cursor.fetchall()
matched_data = []
for n in neighbors:
    matched_data.append(
        {
            "time": str(n[0]),
            # Handle None values for recent data that might not have labels yet
            "next_return": float(n[1]) if n[1] is not None else 0.0,
            "next_slope_3": float(n[2]) if n[2] is not None else 0.0,
            "next_slope_5": float(n[3]) if n[3] is not None else 0.0,
            "distance": float(n[4]),
            "similarity_score": 1 - float(n[4]),
            "embedding": n[5]
        }
    )

In [44]:
def get_min_max_distinct(items, key='distance'):
    if not items:
        return []
    
    # 1. Sort the list by the chosen metric (e.g., distance)
    sorted_items = sorted(items, key=lambda x: x[key])
    
    # 2. Extract Min and Max
    min_item = sorted_items[1]
    max_item = sorted_items[-1]
    
    # 3. Ensure they are distinct (handle lists with 1 item or identical values)
    # We compare the 'time' or unique ID to ensure we don't return the same object twice if the list is short
    if min_item['time'] == max_item['time']:
        return [min_item]
    
    selected = {min_item['time']: min_item, max_item['time']: max_item}
    distinct_elements = list(selected.values())
    
    returns = [abs(item['distance']) for item in distinct_elements]
    average_return = sum((returns)) / len(returns)
    
    return [min_item, max_item, average_return]

# Execution
min_distance, max_distance, average_return = get_min_max_distinct(matched_data)

In [45]:
min_distance['distance'], max_distance['distance'], average_return

(0.47703997642768836, 0.5327028737971025, 0.5048714251123954)

In [46]:
image_b64 = ai.plot_patterns_to_base64(
    current_vec=target_vec,
    rag_matches=[min_distance]
)

min_dist = min_distance['distance']
image_data = base64.b64decode(image_b64)
min_path = os.path.join('check_outputs', 'min', f"{str(min_dist)}.png")
with open(min_path, "wb") as file:
    file.write(image_data)

In [47]:
image_b64 = ai.plot_patterns_to_base64(
    current_vec=target_vec,
    rag_matches=[max_distance]
)

max_dist = max_distance['distance']
image_data = base64.b64decode(image_b64)
max_path = os.path.join('check_outputs', 'max', f"{str(max_dist)}.png")
with open(max_path, "wb") as file:
    file.write(image_data)

In [48]:
image_b64 = ai.plot_patterns_to_base64(
    current_vec=target_vec,
    rag_matches=matched_data
)

average_dist = average_return
image_data = base64.b64decode(image_b64)
average_path = os.path.join('check_outputs', 'average', f"{str(average_dist)}.png")
with open(average_path, "wb") as file:
    file.write(image_data)

# Check filter distnace

In [857]:
is_correct_3 = []
is_correct_5 = []
avg_distance_lst = []

In [858]:
symbol = "ADAUSDT"
time_column="time"
vector_column="embedding"
table_name="market_pattern"

top_k=10


In [969]:
db = PostgreSQLDB()
db.set_connection()

ai = PatternAI(
    symbol=symbol,
    timeframe="15m",
    model="MOCKUP",
    vector_window=60,
)

raw_sql = f"""
    SELECT {vector_column} 
    FROM {table_name}
    where time = '2026-01-04 02:45:00+07:00' 
    order by random()
    limit 1
"""
db.cursor.execute(raw_sql)
row = db.cursor.fetchone()
target_vec = row[0]

if hasattr(target_vec, 'tolist'):
    target_vec = target_vec.tolist()
    
if isinstance(target_vec, str):
    actual_vector = json.loads(target_vec)
else:
    actual_vector = target_vec

2026-01-04 03:00:40,455 - PostgreSQLDB - INFO - Initialize database with db_name: prod || user: postgres


Creating UTC logger with level: 20
Creating UTC logger with level: 20
Creating UTC logger with level: INFO
Creating UTC logger with level: INFO


In [970]:
query_neighbors = f"""
    SELECT 
        {time_column}, 
        next_return, 
        next_slope_3, 
        next_slope_5,
        {vector_column} <=> '{actual_vector}' as distance,
        {vector_column}
    FROM {table_name}
    WHERE symbol = '{symbol}'
    AND {vector_column} IS NOT NULL
    AND {time_column} < '2026-01-01 00:00:00'::timestamptz
    ORDER BY distance ASC
    LIMIT {top_k}
"""

db.cursor.execute(query_neighbors)

neighbors = db.cursor.fetchall()
matched_data = []
for n in neighbors:
    matched_data.append(
        {
            "time": str(n[0]),
            # Handle None values for recent data that might not have labels yet
            "next_return": float(n[1]) if n[1] is not None else 0.0,
            "next_slope_3": float(n[2]) if n[2] is not None else 0.0,
            "next_slope_5": float(n[3]) if n[3] is not None else 0.0,
            "distance": float(n[4]),
            "similarity_score": 1 - float(n[4]),
            "embedding": n[5]
        }
    )

In [971]:
min_distance, max_distance, average_distance = get_min_max_distinct(matched_data)
min_distance, max_distance, average_distance

({'time': '2024-06-18 14:00:00+00:00',
  'next_return': 0.004568664337543765,
  'next_slope_3': -0.0045478865703584975,
  'next_slope_5': -0.0017388978063135527,
  'distance': 0.4669376882950914,
  'similarity_score': 0.5330623117049086,
  'embedding': '[-0.10237462,0.02859803,0.05043299,0.006500974,-0.19162764,-0.5692374,0.6252038,-0.28195858,-1.5623729,-6.547215,2.5915754,-0.12113206,1.3666162,-0.0010430464,0.5840726,-0.0940824,0.25638914,0.53549653,0.23229338,0.1393739,0.30160186,0.18560436,-0.069121264,0.37083784,0.23168306,0.0699788,-0.30062243,0.39424607,-0.09230977,-0.23239663,0.04628815,0.3023874,0.2555739,0.30169967,0.023449628,0.32464382,-0.069121264,0.4863539,0.13925202,-0.068683,-0.39418367,0.18593498,-0.13991997,-0.39825562,0.37375456,0.13954669,-0.047621395,-0.30640966,-0.16668338,0.11616702,0.6340329,-0.49618495,-0.23906279,-0.026327755,0.7791756,-0.14370468,-0.31074145,0.25870028,-0.33598334,0.9709113]'},
 {'time': '2024-12-05 13:30:00+00:00',
  'next_return': 0.0007425

In [972]:
minus_slope_3 = []
minus_slope_5 = []

plus_slope_3 = []
plus_slope_5 = []

matched_data = matched_data[1:]

for element in matched_data:
    
    # Slope 3
    if element['next_slope_3'] < 0.0:
        minus_slope_3.append(element['next_slope_3'])
    elif element['next_slope_3'] > 0.0:
        plus_slope_3.append(element['next_slope_3'])
        
    if element['next_slope_5'] < 0.0:
        minus_slope_5.append(element['next_slope_5'])
    elif element['next_slope_5'] > 0.0:
        plus_slope_5.append(element['next_slope_5'])
        
actual_slope_3 = matched_data[0]['next_slope_3']
actual_slope_5 = matched_data[0]['next_slope_5']
        
minus_slope_3, minus_slope_5 , plus_slope_3, plus_slope_5

([-0.0045478865703584975, -0.0007248477819657884, -1.0909573915637432e-19],
 [-0.0017388978063135527,
  -0.0008988112496375736,
  -5.1948051948046164e-05,
  -0.0002614918799889914,
  -0.0016572905580051558,
  -0.0022590485612993725],
 [0.003767491926803017,
  0.001182033096926693,
  0.00038759689922476267,
  0.0014450867052023058,
  0.0019005625665197326,
  0.0004122351389231979],
 [0.0007965554359526213, 0.0003152088258470891, 0.001162790697674405])

In [973]:
assert len(minus_slope_3) + len(plus_slope_3) == top_k - 1
assert len(minus_slope_5) + len(plus_slope_5) == top_k - 1

In [974]:
# Determine Predictions (True if Positive, False if Negative)
pred_3_is_positive = len(plus_slope_3) > len(minus_slope_3)
pred_5_is_positive = len(plus_slope_5) > len(minus_slope_5)

# Determine Actuals
actual_3_is_positive = actual_slope_3 > 0
actual_5_is_positive = actual_slope_5 > 0

# Compare
# This returns True if both are True, or both are False.
is_correct_3.append(pred_3_is_positive == actual_3_is_positive)
is_correct_5.append(pred_5_is_positive == actual_5_is_positive)
avg_distance_lst.append(average_distance)

In [975]:
import pandas as pd

df = pd.DataFrame(
    data={
        "distance_avg": avg_distance_lst,
        "is_correct_3": is_correct_3,
        "is_correct_5": is_correct_5,
        
    }
)

In [976]:
# Calculate basic accuracy (percentage of True values)
accuracy_3 = df['is_correct_3'].mean()
accuracy_5 = df['is_correct_5'].mean()

print(f"Window 3 Accuracy: {accuracy_3:.2%}")
print(f"Window 5 Accuracy: {accuracy_5:.2%}")

Window 3 Accuracy: 75.00%
Window 5 Accuracy: 41.67%


In [977]:
# Group by correctness to see the average distance for Wins vs. Losses
print("\n--- Impact of Distance on Window 3 ---")
print(df.groupby('is_correct_3')['distance_avg'].describe()[['count', 'mean', 'std']])

print("\n--- Impact of Distance on Window 5 ---")
print(df.groupby('is_correct_5')['distance_avg'].describe()[['count', 'mean', 'std']])


--- Impact of Distance on Window 3 ---
              count      mean       std
is_correct_3                           
False           3.0  0.487107  0.009660
True            9.0  0.489115  0.008195

--- Impact of Distance on Window 5 ---
              count      mean       std
is_correct_5                           
False           7.0  0.490698  0.008956
True            5.0  0.485695  0.006684


In [978]:
# Create "bins" for distance to see accuracy at different levels
# Example: 0.0-0.2, 0.2-0.4, etc.
df['distance_bucket'] = pd.cut(df['distance_avg'], bins=5)

# Check accuracy per bucket
analysis = df.groupby('distance_bucket')[['is_correct_3', 'is_correct_5']].mean()
print(analysis)

                 is_correct_3  is_correct_5
distance_bucket                            
(0.48, 0.485]        0.666667           0.5
(0.485, 0.491]       1.000000           0.0
(0.491, 0.496]       1.000000           1.0
(0.496, 0.501]       0.000000           0.0
(0.501, 0.507]       1.000000           0.0


/var/folders/m8/n99fy2ds2j70c3fm9g4zg65w0000gp/T/ipykernel_63570/4262022989.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  analysis = df.groupby('distance_bucket')[['is_correct_3', 'is_correct_5']].mean()
